In [101]:
import re
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from textblob import TextBlob
from collections import Counter
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
full_df = pd.read_csv('../DataSet/twcs.csv', nrows=100000)
full_df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [15]:
df = full_df[['text']]
df['text'] = df['text'].astype(str)
df.head()

,text
0,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...
3,@115712 Please send us a Private Message so th...
4,@sprintcare I did.


1. LowerCase conversion

In [18]:
df['text_lower'] = df['text'].str.lower()
df.head()

,text,text_lower
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...
4,@sprintcare I did.,@sprintcare i did.


2. Remove punctuations

In [34]:
print(string.punctuation)
df['text_wo_punct'] = df['text_lower'].str.replace('[^\w\s]', '')

# def remove_punct(text):
#     return text.translate(str.maketrans('', '', string.punctuation))
# df['text_wo_punct'] = df['text_lower'].apply(lambda text: remove_punct(text))

df.head()

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


,text,text_lower,text_wo_punct
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did


3. Remove Stopwords

In [36]:
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [39]:
stop = stopwords.words('english')
df['text_wo_stopwords'] = df['text_wo_punct'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# def remove_stopwords(text):
#     return " ".join([word for word in str(text).split() if word not in stop])
# df['text_wo_stopwords'] = df['text_wo_punct'].apply(lambda text: remove_stopwords(text))

df.head()

,text,text_lower,text_wo_punct,text_wo_stopwords
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare


4. Remove Frequent words

In [54]:
cnt = Counter()
for text in df['text_wo_stopwords'].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('us', 15285),
 ('please', 14185),
 ('dm', 10293),
 ('help', 9192),
 ('hi', 8369),
 ('thanks', 7807),
 ('get', 7395),
 ('sorry', 6919),
 ('im', 6454),
 ('amazonhelp', 5917)]

In [58]:
# FreqWords = set([w for w,wc in cnt.most_common(10)])
FreqWords = [w for w,wc in cnt.most_common(10)]
print(FreqWords)
def remove_freqwords(text):
    return " ".join([word for word in text.split() if word not in FreqWords])

df['text_wo_freqwords'] = df['text_wo_stopwords'].apply(lambda text: remove_freqwords(text))
df.head()

['us', 'please', 'dm', 'help', 'hi', 'thanks', 'get', 'sorry', 'im', 'amazonhelp']


,text,text_lower,text_wo_punct,text_wo_stopwords,text_wo_freqwords
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click messa...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare


5. Remove Rare words

In [92]:
cnt.most_common()[-11:-1]

[('httpstcoehp8tcbeds', 1),
 ('144295', 1),
 ('httpstcoklebhaiotk', 1),
 ('7109', 1),
 ('7110', 1),
 ('144296', 1),
 ('httpstcosxccepn7bl', 1),
 ('230k', 1),
 ('mailtemplate', 1),
 ('therelets', 1)]

In [95]:
n_rare_words = 10
RareWords = [w for w,wc in cnt.most_common()[:-n_rare_words-1:-1]]
print(RareWords)
def remove_rarewords(text):
    return " ".join([word for word in text.split() if word not in RareWords])

df['text_wo_rarewords'] = df['text_wo_freqwords'].apply(lambda text: remove_rarewords(text))
df.head()

['phone131027', 'therelets', 'mailtemplate', '230k', 'httpstcosxccepn7bl', '144296', '7110', '7109', 'httpstcoklebhaiotk', '144295']


,text,text_lower,text_wo_punct,text_wo_stopwords,text_wo_freqwords,text_wo_rarewords
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click messa...,115712 send private message assist click messa...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare


6. Stemming

In [97]:
stemmer = SnowballStemmer("english")
df['text_stemmed'] = df['text_wo_rarewords'].apply(lambda text: " ".join([stemmer.stem(word) for word in text.split()]))
# def stem_words(text):
#     return " ".join([stemmer.stem(word) for word in text.split()])
# df['text_stemmed'] = df['text_wo_rarewords'].apply(lambda text: stem_words(text))
df.head()

,text,text_lower,text_wo_punct,text_wo_stopwords,text_wo_freqwords,text_wo_rarewords,text_stemmed
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose,sprintcar propos
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcar sent sever privat messag one respond...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click messa...,115712 send private message assist click messa...,115712 send privat messag assist click messag ...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare,sprintcar


7. Lemmatization

In [99]:
lemma = WordNetLemmatizer()
df['text_lemmatized'] = df['text_wo_rarewords'].apply(lambda text: " ".join([lemma.lemmatize(word) for word in text.split()]))
df.head()

,text,text_lower,text_wo_punct,text_wo_stopwords,text_wo_freqwords,text_wo_rarewords,text_stemmed,text_lemmatized
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose,sprintcar propos,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcar sent sever privat messag one respond...,sprintcare sent several private message one re...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click messa...,115712 send private message assist click messa...,115712 send privat messag assist click messag ...,115712 send private message assist click messa...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare,sprintcar,sprintcare


In [102]:
lemma = WordNetLemmatizer()
wordnet_map = {'N':wordnet.NOUN, 'V':wordnet.VERB, 'J':wordnet.ADJ, 'R':wordnet.ADV}
def lemmatized_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemma.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
df['text_lemmatized_noun'] = df['text_wo_rarewords'].apply(lambda text: lemmatized_words(text))
df.head()

,text,text_lower,text_wo_punct,text_wo_stopwords,text_wo_freqwords,text_wo_rarewords,text_stemmed,text_lemmatized,text_lemmatized_noun
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose,sprintcar propos,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcar sent sever privat messag one respond...,sprintcare sent several private message one re...,sprintcare send several private message one re...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click messa...,115712 send private message assist click messa...,115712 send privat messag assist click messag ...,115712 send private message assist click messa...,115712 send private message assist click messa...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare,sprintcar,sprintcare,sprintcare


In [103]:
df.head(25)

,text,text_lower,text_wo_punct,text_wo_stopwords,text_wo_freqwords,text_wo_rarewords,text_stemmed,text_lemmatized,text_lemmatized_noun
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose,sprintcare propose,sprintcar propos,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...,sprintcar sent sever privat messag one respond...,sprintcare sent several private message one re...,sprintcare send several private message one re...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 send private message assist click messa...,115712 send private message assist click messa...,115712 send privat messag assist click messag ...,115712 send private message assist click messa...,115712 send private message assist click messa...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare,sprintcare,sprintcare,sprintcar,sprintcare,sprintcare
5,@115712 Can you please send us a private messa...,@115712 can you please send us a private messa...,115712 can you please send us a private messag...,115712 please send us private message gain det...,115712 send private message gain details account,115712 send private message gain details account,115712 send privat messag gain detail account,115712 send private message gain detail account,115712 send private message gain detail account
6,@sprintcare is the worst customer service,@sprintcare is the worst customer service,sprintcare is the worst customer service,sprintcare worst customer service,sprintcare worst customer service,sprintcare worst customer service,sprintcar worst custom servic,sprintcare worst customer service,sprintcare bad customer service
7,@115713 This is saddening to hear. Please shoo...,@115713 this is saddening to hear. please shoo...,115713 this is saddening to hear please shoot ...,115713 saddening hear please shoot us dm look kc,115713 saddening hear shoot look kc,115713 saddening hear shoot look kc,115713 sadden hear shoot look kc,115713 saddening hear shoot look kc,115713 sadden hear shoot look kc
8,@sprintcare You gonna magically change your co...,@sprintcare you gonna magically change your co...,sprintcare you gonna magically change your con...,sprintcare gonna magically change connectivity...,sprintcare gonna magically change connectivity...,sprintcare gonna magically change connectivity...,sprintcar gonna magic chang connect whole famili,sprintcare gonna magically change connectivity...,sprintcare gonna magically change connectivity...
9,@115713 We understand your concerns and we'd l...,@115713 we understand your concerns and we'd l...,115713 we understand your concerns and wed lik...,115713 understand concerns wed like please sen...,115713 understand concerns wed like send direc...,115713 understand concerns wed like send direc...,115713 understand concern wed like send direct...,115713 understand concern wed like send direct...,115713 understand concern wed like send direct...


8. Remove Emojis

In [111]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
remove_emoji('Complete Version of remove Emojis ✍ 🌷 📌 👈🏻 🖥♀')

'Complete Version of remove Emojis     '

In [108]:
pip install emoji

In [110]:
import emoji

def remove_emojis(text: str) -> str:
    return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)

remove_emoji('Complete Version of remove Emojis ✍ 🌷 📌 👈🏻 🖥♀')

'Complete Version of remove Emojis     '

9. Remove URLs

In [112]:
def remove_url(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

remove_url('checkout www.h2o.ai for more info')

'checkout  for more info'

10. Remove HTML tags

In [119]:
def remove_html(text):
    html_pattern = re.compile(r'<.*?>')
    return html_pattern.sub(r'', text)

In [121]:
text = """<div>
<h1>Title</h1>
<p>A long text........ </p>
<a href=""> a link </a>
</div>"""

print(remove_html(text))


Title
A long text........ 
 a link 

